## REDSEA python version for Steinbock

Edited substantially and re-organized from the original REDSEA python version 0.0.1 file in this folder (see that file for details on the original python version).

Edited by Ben Caiello, with the intent of allowing it to work with current DeepCell segmentation masks / the steinbock pipeline. The algorithm is not identical, as that is not possible (I think) given the differenes in the masks of old DeepCell and new DeepCell (+ / - zero boundaries changes the effective pixel width of the boundary measurement step).

Dataset I had been using for CD3 / CD20 compensation: https://zenodo.org/records/8023452

Passed through steinbock to derive .tiffs and masks, then fed into this script

The directory structure required for the script as-written is the one naturally produced by steinbock:

A master directory with two folders: \img & \masks - each containing .tiffs with the original images and the DeepCell generated masks, repectively, with matching file names - and 1 .csv file (panel.csv). These are all naturally produced by steinbock when it is run.

The script has not been thoroughly tested, but should produce outputs and seems to be doing what it is supposed to with the limited testing so far.


Some (potential and certain) differences in the algorithm to note:
    1. Since the masks format is different with DeepCell now (no cell-cell padding with 0's), the algorithm is made to find border px by looking for >1 segmentation label (indicating >1 cell or cell + background boundary)
    2. Also, because of the lack of padding, the effective distances and sizes of the diamond / square boder px identification step is altered

In [1]:
# Package Imports
import PIL
from PIL import Image, ImageSequence, ImageOps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage 
import skimage.measure
import skimage.morphology
import glob
from scipy.io import loadmat
import time

import tifffile
import os

In [2]:
## Functions (under normal operations / no errors, minimize this cell to see things more easily)
## If this actually works as intended, these should probably be moved to a separate file / turned into a mini-python library

# The first two functions are from the original script
# helper function 1
def ismember(a, b):
    bind = {}
    for i, elt in enumerate(b):
        if elt not in bind:
            bind[elt] = i
    return [bind.get(itm, None) for itm in a]  # None can be replaced by any other "not in b" value

# helper function 2

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    '''This prints a progress bar'''
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()


## From now these are BPC edited functions
def file_reader(steinbockPath, file_name, ends_with_tiff = False):
    '''
    This function reads in the panel file, the image, and its matching segmentation image from your Steinbock output folder.

    args:
    steinbockPath  -- this is the directory path to your steinbock output
    file_name -- this is the name of the ROI / image to be read.
    ends_with_tiff (default = False) -- this determines whether you include the .tiff suffix in the file_name. By default, do not include .tiff

    returns:
    clusterChannels -- the channels from the panel file kept by the steinbock program
    Full_Tiff -- a multi-dimensional numpy array representing all of the channels of the .tiff image
    Segmentation -- a 2D numpy array representing the segmentation by DeepCell
    '''
    ## panel file
    if ends_with_tiff == True:
        file_name = file_name
    elif ends_with_tiff == False:
        file_name = file_name + ".tiff"
    
    massDS_path = steinbockPath + '\\panel.csv' # csv file location, need
    massDS = pd.read_csv(massDS_path) # read the mass csv
    clusterChannelsDataFrame = massDS[massDS['keep'] == 1]
    clusterChannels = clusterChannelsDataFrame.reset_index().index.values
    
    # Tiff file
    pathTiff = steinbockPath + '\\img\\' + file_name #  tiff location, links to a single .tiff
    array_list=[]
    for channel in clusterChannels:
        t=tifffile.imread(pathTiff)[channel]
        array_list.append(t)
    Full_Tiff=np.stack(array_list,axis=2) # count matrices in the image

    ## Segmentation file
    pathMat = steinbockPath + '\\masks\\' + file_name   # corresponding .tiff's mask
    Segmentation = tifffile.imread(pathMat).astype('int')

    return clusterChannelsDataFrame, Full_Tiff, Segmentation

def cell_statistics(clusterChannelsDataFrame, Full_Tiff, Segmentation):
    ''' This recovers key statistics from the files: cell / channel number, cell sizes & data, and image dimensions'''
    clusterChannels = clusterChannelsDataFrame.reset_index().index.values
    channelNum = len(clusterChannels) # how many channels
    
    cellNum = np.max(Segmentation) # how many labels
    stats = skimage.measure.regionprops(Segmentation) # get the regional props for all the labels
    
    ### make empty container matrices
    data = np.zeros((cellNum,channelNum))
    dataScaleSize = np.zeros((cellNum,channelNum))
    cellSizes = np.zeros((cellNum,1))

    # this part extract counts data from the whole cell regions, for each individual cells etc
    
    for i in range(cellNum): # for each cell (label)
        label_counts=[Full_Tiff[coord[0],coord[1],:] for coord in stats[i].coords] # all channel count for this cell
        data[i,0:channelNum] = np.sum(label_counts, axis=0) #  sum the counts for this cell
        dataScaleSize[i,0:channelNum] = np.sum(label_counts, axis=0) / stats[i].area # scaled by size
        cellSizes[i] = stats[i].area # cell sizes
    
    [rowNum, colNum] = Segmentation.shape      #could also have been Full_Tiff.shape
    
    return cellNum, channelNum, data, cellSizes, dataScaleSize, rowNum, colNum


def cell_cell_matrix(Segmentation, cellNum, rowNum, colNum,  REDSEAChecker = 1):
    ''' Identifies cell-cell contacts, creating a matrix representing the number and identity of each cell's contact with other cells.

    args:
    Segmentation -- this is the numpy array representation of the DeepCell segmentation
    cellNum -- this is the number of cells in the segmentation
    rowNum -- the number of rows of pixels in the image
    colNum -- the number of columns of pixels in the image
    REDSEAChecker -- this determines whether the the algorithm reinforces a cell's number back to itself & compensates (1) or only removes signal with the compensation (0). Defaults to 1. 

    returns:
    cellPairNorm -- this is the cell-cell contact matrix used for compensation
    '''
    cellPairMap = np.zeros(((cellNum + 1),(cellNum + 1))) # cell-cell shared perimeter matrix container
    
    # start looping the mask and produce the cell-cell contact matrix
    for i in range(rowNum):
        if i == 0:   # these conditional statements account for pixels on the edge of the image by shrinking the 3x3 box to smaller dimensions
            a = 0
            c = 2
        elif i == (rowNum - 1):
            a = 1
            c = 1
        else:
            a = 1
            c = 2
        for j in range(colNum):
            if j == 0:
                b = 0
                d = 2
            elif j == (colNum - 1):
                b = 1
                d = 1
            else:
                b = 1
                d = 2
            tempMatrix = Segmentation[i-a:i+c,j-b:j+d] # the 3x3 window, centered on the point i,j
            #print(tempMatrix)
            tempFactors = np.unique(tempMatrix).astype('int') #unique
            #print(tempFactors)
            centerpoint_value = Segmentation[i,j]
            #print(centerpoint_value)
            for k in tempFactors:
                if k != centerpoint_value: # only add to the cellPairMap for the centerpoint pixel -- this prevents multiplicate counting
                    #print("trigger")
                    cellPairMap[centerpoint_value,k] = cellPairMap[centerpoint_value,k] + 1  
        
    # converting the cell cell maps to fraction of cell - cell boundary (not of total cell boundary [!?] -- as in boundary with empty space not counted)
    cellPairNorm = np.zeros(((cellNum+1),(cellNum+1)))
    for i in np.arange(0,len(cellPairMap)):
        cellPairNorm[i] = - (cellPairMap[i] / np.sum(cellPairMap[i]))
    cellPairNorm = cellPairNorm[1:,1:]
    cellPairNorm = cellPairNorm + REDSEAChecker*np.identity(cellNum ) 
    return cellPairNorm

def Cell_Edge_Intensities(Segmentation, cellNum, channelNum, Full_Tiff, rowNum, colNum, elementShape = 2, elementSize = 2):
    '''
    This generates the intensities of each cell's border pixels in every channel for use in the compensation
    '''
    MIBIdataNearEdge1 = np.zeros((cellNum+1,channelNum))
    MIBIdataNearEdgeAvg = np.zeros((cellNum+1,channelNum))
    #initialize progress bar:
    items = list(range(cellNum + 1))
    l = len(items)
    printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 100) # progress bar
    
    # Square or Diamond shape of desired size
    if elementShape==1: # square
        shape=skimage.morphology.square((2*elementSize) + 1)
    elif elementShape==2: # diamond
        shape=skimage.morphology.diamond(elementSize) # create diamond shapte based on elementSize
    else:
        print("Error elementShape Value not recognized.")   

    # iterate over cells, measuring total border intensity
    list_num_border_px = []
    for i in range(cellNum + 1) :
        if i == 0:
            continue
        num_border_px = 0
        [tempRow,tempCol] = np.asarray(Segmentation==i).nonzero()
        [rowNum, colNum] = Segmentation.shape
        # sequence in row not col, should not affect the code
        for j in range(len(tempRow)):
            label_in_shape=[] # empty list in case
            if (tempRow[j] - elementSize) <= -1:   # these conditionals are for ensuring the tempMatrix does not cross the image edge
                a = 0 + tempRow[j]
                c = 1 + elementSize
            elif (tempRow[j] + elementSize) > rowNum - 1:
                a = 0 + elementSize
                c = 1 + rowNum - tempCol[j]
            else:
                a = 0 + elementSize
                c = 1 + elementSize
            if (tempCol[j] - elementSize) <= -1:
                b = 0 + tempCol[j]
                d = 1 + elementSize
            elif (tempCol[j] + elementSize) > colNum - 1:
                b = 0 + elementSize
                d = 1 + colNum - tempCol[j]
            else:
                b = 0 + elementSize
                d = 1 + elementSize
            tempMatrix = Segmentation[tempRow[j]-a:tempRow[j]+c,tempCol[j]-b:tempCol[j]+d] # the 3x3 window, centered on the point i,j
            if elementShape == 2:
                [dim1, dim2] = tempMatrix.shape
                reducedShape = shape[:dim1,:dim2].astype('bool')
                reducedMatrix = tempMatrix[reducedShape]   # the reduced shape /  matrix lines are there to accommodate the diamond shape
                tempFactors = np.unique(reducedMatrix).astype('int')
            else:  # no need for reduction if using the square shape
                tempFactors = np.unique(tempMatrix).astype('int')
            if len(tempFactors) > 1:
                num_border_px += 1
                MIBIdataNearEdge1[i,:] = MIBIdataNearEdge1[i,:] + Full_Tiff[tempRow[j],tempCol[j],:]
        MIBIdataNearEdgeAvg[i,:] =  MIBIdataNearEdge1[i,:] / num_border_px
        list_num_border_px.append(num_border_px)
        # Update Progress Bar
        if ((i % 500) == 0) or (i == (cellNum - 1)):
            printProgressBar(i + 1, l, prefix = 'Progress:', suffix = 'Complete', length = 100)
    MIBIdataNearEdge1 = MIBIdataNearEdge1[1:,:]
    MIBIdataNearEdgeAvg = MIBIdataNearEdgeAvg[1:,:]
    return MIBIdataNearEdge1, MIBIdataNearEdgeAvg, list_num_border_px
            
def CalculateREDSEA(MIBIdataNearEdge1, cellPairNorm, data, dataScaleSize, normChannels, clusterChannelsDataFrame, cellNum, cellSizes, Border_average = False, MIBIdataNearEdgeAvg = 0):  
    '''
    Takes the cell-cell matrix and border intensity matrix and calculates compensation.

    args:
    MIBIdataNearEdge1 -- cell border intensity matrix
    cellPairNorm -- cell-cell contact matrix
    data -- raw intensity measured for each cell in each channel
    dataScaleSize -- the raw intensities of each normalized to cell size
    normChannels -- the channels you wish to compensate with REDSEA
    clusterChannelsDataFrame -- a dataframe of the channels in the dataset
    cellNum -- the number of cell in the image
    cellSizes -- the sizes of each cell
    Border_average -- whether to use the raw border intensity of each cell to compensate the raw intensity of that cell (default), or the size normalized border intensity and size normalized whole cell intensity
    MIBIdataNearEdgeAvg -- cell border intensity matrix, but with border intensities normalized by number of pixels (size of border regions). Only use with Border_average = True
    
    returns:
    Four pandas dataframes, ready for export to csv's or examination in jupyter lab:
    dataCells -- uncompensated (pre-REDSEA) cell intensity values
    dataScaleSizeCells -- uncompensated cell intensities, normalized by cell size
    dataCompenCells -- compensated cell intensity values
    dataCompenScaleSizeCells -- compensated cell intensity values, normalized by cell sizes
    '''
    clusterChannels = clusterChannelsDataFrame.reset_index().index.values
    normChannelsInds = ismember(normChannels,clusterChannels)
    channelNormIdentity = np.zeros((len(clusterChannels),1))
    # make a flag for compensation
    for i in range(len(normChannelsInds)):
            channelNormIdentity[normChannelsInds[i]] = 1 
    if Border_average == False:
        MIBIdataNorm2 = np.transpose(np.dot(np.transpose(MIBIdataNearEdge1[:,:]),cellPairNorm))
        #this is boundary signal subtracted by cell neighbor boundary
        MIBIdataNorm2 = MIBIdataNorm2 + data # reinforce onto the whole cell signal (original signal)
        
        MIBIdataNorm2[MIBIdataNorm2<0] = 0 # clear out the negative ones
        # flip the channelNormIdentity for calculation
        rev_channelNormIdentity=np.ones_like(channelNormIdentity)-channelNormIdentity
        # composite the normalized channels with non-normalized channels
        # MIBIdataNorm2 is the matrix to return
        MIBIdataNorm2 = data * np.transpose(np.tile(rev_channelNormIdentity,(1,cellNum))) + MIBIdataNorm2 * np.transpose(np.tile(channelNormIdentity,(1,cellNum)))
        
        # the function should return 4 variables
        dataCells = data
        dataScaleSizeCells = dataScaleSize
        dataCompenCells = MIBIdataNorm2
        dataCompenScaleSizeCells = MIBIdataNorm2 / cellSizes
    elif Border_average == True:
        MIBIdataNorm2 = np.transpose(np.dot(np.transpose(MIBIdataNearEdgeAvg),cellPairNorm))
        MIBIdataNorm2 = MIBIdataNorm2 + dataScaleSize
        MIBIdataNorm2[MIBIdataNorm2<0] = 0
        rev_channelNormIdentity=np.ones_like(channelNormIdentity)-channelNormIdentity
        MIBIdataNorm2 = data * np.transpose(np.tile(rev_channelNormIdentity,(1,cellNum))) + (MIBIdataNorm2 * np.transpose(np.tile(channelNormIdentity,(1,cellNum))) * cellSizes)
        dataCells = dataScaleSize * cellSizes
        dataScaleSizeCells = dataScaleSize
        dataCompenCells = MIBIdataNorm2 
        dataCompenScaleSizeCells = MIBIdataNorm2 / cellSizes
    list_of_arrays = [dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells]
    for jj,j in enumerate(list_of_arrays):
        labelVec = [i for i in np.arange(1,cellNum + 1,1)]
        cellSizesVec_flat = [item for sublist in cellSizes for item in sublist] # flat the list
        dataL = pd.DataFrame({'Object':labelVec})
        dataCells_df=pd.DataFrame(j)
        list_of_columns = []
        for i in clusterChannelsDataFrame["name"]:
            list_of_columns.append(i)
        dataCells_df.columns = list_of_columns
        if jj == 0:
            dataCells = pd.concat((dataL,dataCells_df),axis=1)
        if jj == 1:
            dataScaleSizeCells = pd.concat((dataL,dataCells_df),axis=1)
        if jj == 2:
            dataCompenCells = pd.concat((dataL,dataCells_df),axis=1)
        if jj == 3:
            dataCompenScaleSizeCells = pd.concat((dataL,dataCells_df),axis=1)
    return dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells

def Outputter(data_in, file_name, pathResults):
    data_in.to_csv(pathResults + '\\' + file_name + ".csv", index = False)


In [3]:
######## Edit these for your run!
# file locations
steinbockPath = 'C:\\Users\\caiello\\Desktop\\RedSEA practice\\practice' # main folder must contain \img and \masks folders and a panel.csv file (matching steinbock output)

# or set file_name manually
file_name = 'RNANeg_Tonsil_003'  #do not include the .tiff suffix! Or, alternatively, include .tiff, but set the ends_with_tiff argument in the file_reader function to True!

#  select which channel to normalize
normChannels = [13,18]    

# parameters for compensation (change as desired)
REDSEAChecker = 1 # 1 means subtract + reinforce
elementShape = 1 # star == 2, square == 1
elementSize = 1 # star or square expansion size

# output path
pathResults = steinbockPath + '/intensities' # output location, set as intensities here to match steinbock output
try:
    os.listdir(pathResults)
except:
    os.mkdir(pathResults)

alt_out_path = "\\an\\alternate\folder\\in\\which\\to\\put\\your\\non-scaled\\and\\non-compensated\\data" ## use if you want the non-compensated / non-scaled data, but don't want them crowding the main steinbock intensities folder
######### if you use alt_path, manually make the directory in file explorer!!!

In [4]:
## Run the Program! Single file run:
clusterChannelsDataFrame, Full_Tiff, Segmentation = file_reader(steinbockPath, file_name)
cellNum, channelNum, data, cellSizes, dataScaleSize, rowNum, colNum = cell_statistics(clusterChannelsDataFrame, Full_Tiff, Segmentation)
cellPairNorm = cell_cell_matrix(Segmentation, cellNum, rowNum, colNum,  REDSEAChecker = 1)
MIBIdataNearEdge1, MIBIdataNearEdgeAvg, list_num_border_px = Cell_Edge_Intensities(Segmentation, cellNum, channelNum, Full_Tiff, rowNum, colNum, elementShape = elementShape, elementSize = elementSize)
dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1, cellPairNorm, data, dataScaleSize, normChannels, clusterChannelsDataFrame, cellNum, cellSizes, Border_average = True, MIBIdataNearEdgeAvg = MIBIdataNearEdgeAvg)

Outputter(dataScaleSizeCells, file_name = file_name + "_pre_REDSEA_scaled", pathResults = pathResults)
Outputter(dataCompenScaleSizeCells, file_name, pathResults = pathResults)

In [62]:
## Run the Program! Whole folder of images at once!
file_list = []
for i in os.listdir(steinbockPath + "\\img"):
    i = i.replace(".tiff","")
    file_list.append(i) 

for i in file_list:           # iterate over files in directory
    file_name = i
    print("Starting RedSEA compensation of file: " + file_name)
    clusterChannelsDataFrame, Full_Tiff, Segmentation = file_reader(steinbockPath, file_name)
    cellNum, channelNum, data, cellSizes, dataScaleSize, rowNum, colNum = cell_statistics(clusterChannelsDataFrame, Full_Tiff, Segmentation)
    cellPairNorm = cell_cell_matrix(Segmentation, cellNum, rowNum, colNum,  REDSEAChecker = 1)
    MIBIdataNearEdge1, MIBIdataNearEdgeAvg, list_num_border_px = Cell_Edge_Intensities(Segmentation, cellNum, channelNum, Full_Tiff, rowNum, colNum, elementShape = elementShape, elementSize = elementSize)
    dataCells, dataScaleSizeCells, dataCompenCells, dataCompenScaleSizeCells = CalculateREDSEA(MIBIdataNearEdge1, cellPairNorm, data, dataScaleSize, normChannels, clusterChannelsDataFrame, cellNum, cellSizes, Border_average = False, MIBIdataNearEdgeAvg = 0)
    ################ Change / add Outputter calls if you want to write dataframes besides the scaled cells and compensated scaled cells:  
    Outputter(dataScaleSizeCells, file_name = file_name + "_pre_REDSEA_scaled", pathResults = pathResults)
    Outputter(dataCompenScaleSizeCells, file_name, pathResults = pathResults)


Starting RedSEA compensation of file: RNANeg_Tonsil_001
Starting RedSEA compensation of file: RNANeg_Tonsil_002███████████████████████████████████████████████████████-| 100.0% Complete
Starting RedSEA compensation of file: RNANeg_Tonsil_003███████████████████████████████████████████████████████-| 100.0% Complete
Starting RedSEA compensation of file: RNANeg_Tonsil_004███████████████████████████████████████████████████████-| 100.0% Complete


In [5]:
##### This cell I've been using for testing aspects of the code / data

######### This little code block prints the ratio of border pixels to total cell size in each cell
## With my data I was seeing cells that looked like they were ALL border pixels -- which kind of ruins the point of only selecting border pixel intensities
## If you see this effect, consider reducing the elementSize parameter / using the diamond elementShape (elementShape = 2) to try to classify fewer pixels as border pixels
border_px_percent = []
for i in range(len(cellSizes)):
    border_px_percent.append(list_num_border_px[i] / cellSizes[i])
cell_border_percent_df = pd.DataFrame(border_px_percent, columns = ["ratio"])
display(cell_border_percent_df)
display(cell_border_percent_df.mean())

#dataCompenScaleSizeCells

,ratio
0,0.500000
1,0.488372
2,0.583333
3,0.481481
4,0.395833
...,...
7212,1.000000
7213,0.483871
7214,1.000000
7215,0.875000


ratio    0.444672
dtype: float64